# Дать ответ на 4 вопроса, объеденив данные в один файл.
#### 1. Между какими событиями наибольший шаг, указать пару идентификаторов этих событий
#### 2. Интервал времени до наступления какого события показывает наибольшй разброс?
#### 3. Какая задача прогнозирования выполняется дольше всего,название задачи?
#### 4. Перечислить forecastMarker которые не завершились успешно

Даны 3 лог файла от внутреннего сервиса прогнозируюшего временные ряды. 1 файл 1 день. <br>
События записываются по мере того как они случаются.<br>
forecasMarker - идентификатор задачи прогнозирования<br>
event_id - идентификатор события(если произошло 31 событие задача прогнозирования прошла успешно)<br>
session_id - идентификатор исполнения конкретной задачи.<br>

In [1]:
# loading in modules
import sqlite3
import pandas as pd
# creating file path
dbfile = 'logs_20220912.db'
dbfile2 = 'logs_20220913.db'
dbfile3 = 'logs_20220914.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)
con2 = sqlite3.connect(dbfile2)
con3 = sqlite3.connect(dbfile3)
# creating cursor
cur = con.cursor()
cur2 = con2.cursor()
cur3 = con3.cursor()

# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
print(table_list)

[('event_name',), ('sqlite_sequence',), ('events',), ('sent_alarms',)]


### Извлекаем информацию из трех файлов

In [2]:
base = [dbfile, dbfile2, dbfile3]
events = []
events1 = []
events2 = []
ev = [events, events1, events2]

for i in base:
    for j in ev:
        con = sqlite3.connect(i)
        cur = con.cursor()
        for row in cur.execute("SELECT * FROM 'events'"):
            j.append(row)       
       
    

events = pd.DataFrame(events) 
events = events.set_index(0)
events.columns = ['forecastMarker','request_time','event_id','sender','context','session_id']
eve = events.drop(['context',], axis = 1)
eve.head(6)   

,forecastMarker,request_time,event_id,sender,session_id
0,,,,,
1,SelfService/RUS/turnover,2022-09-12 11:30:11.106628,1,test_system,6f4277f6df57c4f4e90779966522c876
2,SelfService/RUS/turnover,2022-09-12 11:30:11.111359,29,test_system,6f4277f6df57c4f4e90779966522c876
3,SelfService/RUS/turnover,2022-09-12 11:30:11.115989,2,test_system,6f4277f6df57c4f4e90779966522c876
4,SelfService/RUS/turnover,2022-09-12 11:30:12.761754,3,test_system,6f4277f6df57c4f4e90779966522c876
5,SelfService/RUS/turnover,2022-09-12 11:30:12.770134,5,test_system,6f4277f6df57c4f4e90779966522c876
6,SelfService/RUS/turnover,2022-09-12 11:30:12.780523,6,test_system,6f4277f6df57c4f4e90779966522c876


In [3]:
summ = len(eve.index)
summ #Проверка на соединение данных(Проверял кол-во строк в ручную)

44731

### Извлекаем информацию event_name

In [4]:
event = []
for row in cur.execute("SELECT * FROM 'event_name'"):
      event.append(row)   
event = pd.DataFrame(event) 
event = event.set_index(0)      
event.columns = ['event_name']
event.head(6)

,event_name
0,
1,Инициализируем свойства класса
2,Переход в функцию getParametrsAndRegressors
3,Выход из функции getParametrsAndRegressors с п...
4,Выход из функции getParametrsAndRegressors БЕЗ...
5,Записали в свойства класса гиперпараметры и ре...
6,Начинаем читать данные из BQ по полученному SQ...


###  Дадим ответ на первый вопрос. 
#### Между какими событиями наибольший шаг, указать пару идентификаторов этих событий

####  Cортируем по сессии и времени события

In [5]:
full = eve
A1 = full.sort_values(by=['session_id','request_time'])
A1.head()

,forecastMarker,request_time,event_id,sender,session_id
0,,,,,
1723,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.821083,1,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
1724,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.831013,29,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
1725,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.836547,2,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
1726,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.576231,3,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
1727,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.587533,5,weekly_report.saas,0035de8fad557cfe0b187f24b954195f


In [6]:
A1['request_time'] = A1['request_time'].astype("datetime64[ns]")
print('Формат изменен!')

Формат изменен!


In [7]:
A1 = A1.reset_index(drop=True)
A1.head(6)

,forecastMarker,request_time,event_id,sender,session_id
0,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.821083,1,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
1,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.831013,29,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
2,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.836547,2,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
3,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.576231,3,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
4,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.587533,5,weekly_report.saas,0035de8fad557cfe0b187f24b954195f
5,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.594359,6,weekly_report.saas,0035de8fad557cfe0b187f24b954195f


In [8]:
#выводим разницу между событиями
 
rt = []
y = 0
for i in A1['request_time'][1:]:
    step = i - A1['request_time'][y]
    rt.append(step)
    y = y + 1
rt=pd.DataFrame(rt)    
rt.head(6)

,0
0,0 days 00:00:00.009930
1,0 days 00:00:00.005534
2,0 days 00:00:01.739684
3,0 days 00:00:00.011302
4,0 days 00:00:00.006826
5,0 days 00:00:02.008997


In [9]:
#так же можно воспользоваться shift
nr = pd.DataFrame({'0 days 00:00:00.000000'}, index =[0]) #Добавляем пустую строку чтобы индексы совпадали
rt = pd.concat([nr, rt]).reset_index(drop = True)
type(rt)

pandas.core.frame.DataFrame

In [10]:
rt["timedelta"] = pd.to_timedelta(rt[0])
rt.head(6)

,0,timedelta
0,0 days 00:00:00.000000,0 days 00:00:00
1,0 days 00:00:00.009930,0 days 00:00:00.009930
2,0 days 00:00:00.005534,0 days 00:00:00.005534
3,0 days 00:00:01.739684,0 days 00:00:01.739684
4,0 days 00:00:00.011302,0 days 00:00:00.011302
5,0 days 00:00:00.006826,0 days 00:00:00.006826


In [11]:
rt=pd.DataFrame(rt['timedelta'])
ans1 = pd.concat([A1, rt],axis=1)


In [12]:
ans1.loc[ans1['event_id'] == 1, "timedelta"] = '0 days 00:00:00.000000' #заменяем все первые события на 0 


In [13]:
ans1.describe() #Узнаем максимум и выводим его

,event_id,timedelta
count,44731.000000,44731
mean,16.085757,0 days 00:00:00.451142354
std,8.919804,0 days 00:00:01.736237683
min,1.000000,0 days 00:00:00
25%,9.000000,0 days 00:00:00.005598
50%,16.000000,0 days 00:00:00.008131
75%,24.000000,0 days 00:00:00.059087500
max,34.000000,0 days 00:01:46.548210


In [14]:
ans1.loc[ans1['timedelta'] == '0 days 00:01:46.548210']

,forecastMarker,request_time,event_id,sender,session_id,timedelta
13623,weekly_report.care:count_of_jivo_SA,2022-09-14 05:22:40.308128,7,weekly_report.care,4dcab2af4573ced8251fb7433174ba15,0 days 00:01:46.548210


In [15]:
ans1.iloc[13622] #смотрим предыдущий шаг

forecastMarker    weekly_report.care:count_of_jivo_SA
request_time               2022-09-14 05:20:53.759918
event_id                                            6
sender                             weekly_report.care
session_id           4dcab2af4573ced8251fb7433174ba15
timedelta                      0 days 00:00:00.005893
Name: 13622, dtype: object

###  Самый большой шаг между событиями 6 и 7 за 14.09.2022, отклик более 1,5 часa

_

_

###  Дадим ответ на второй вопрос. 
#### Интервал времени до наступления какого события показывает наибольшй разброс?
##### Указать пару id & sender с наибольшим разбросом. Выдвинуть гипотезу с чем связано?

_

#### Разделим столбец с переходным временем из 1 вопроса на части,чтобы работать только с числовой составляющей

In [16]:
import numpy as np
ans1[['NumDays', 'Days', 'Time']] = ans1['timedelta'].astype (str).str.split(' ', expand=True)
ans1.head(5)

,forecastMarker,request_time,event_id,sender,session_id,timedelta,NumDays,Days,Time
0,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.821083,1,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00,0,days,00:00:00
1,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.831013,29,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00.009930,0,days,00:00:00.009930
2,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.836547,2,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00.005534,0,days,00:00:00.005534
3,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.576231,3,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:01.739684,0,days,00:00:01.739684
4,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.587533,5,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00.011302,0,days,00:00:00.011302


In [17]:
ans1['NumDays'].unique()

array(['0'], dtype=object)

In [18]:
ans1['Days'].unique()

array(['days'], dtype=object)

#### Функция перевода микросекунд в целое число

In [19]:
def square_sum(row):
    if row['Time'] != '00:00:00':
        time_obj = datetime.strptime(row['Time'], "%H:%M:%S.%f")
        return (time_obj.hour * 3600000) + (time_obj.minute * 60000) + (time_obj.second * 1000) + (time_obj.microsecond // 1000)
    else:
        return 0

In [20]:
import datetime
from datetime import datetime
ans1['Result'] = ans1.apply(square_sum, axis=1)
ans2 = ans1.drop(ans1[['timedelta', 'NumDays', 'Days']], axis = 1)
ans2.head(5)

,forecastMarker,request_time,event_id,sender,session_id,Time,Result
0,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.821083,1,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,00:00:00,0
1,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.831013,29,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,00:00:00.009930,9
2,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.836547,2,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,00:00:00.005534,5
3,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.576231,3,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,00:00:01.739684,1739
4,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.587533,5,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,00:00:00.011302,11


In [21]:
answ2 = ans2.groupby('session_id')['Result'].agg(list).reset_index() #аггрегируем данные по сесии и времени
answ2

,session_id,Result
0,0035de8fad557cfe0b187f24b954195f,"[0, 9, 5, 1739, 11, 6, 2008, 10, 16, 7, 1690, ..."
1,004df69bde1839cb768d2003ff28ec6c,"[0, 4, 5, 1715, 7, 7, 2536, 5, 7, 5, 1679, 33,..."
2,0090da9b342c3e895cd77bfa1cd747f9,"[0, 7, 4, 1597, 6, 9, 22530, 6, 7, 7, 1722, 32..."
3,00d3543ff8b2d409f3ec4e26cdc246d2,"[0, 17, 7, 1708, 6, 6, 20741, 8, 11, 5, 2136, ..."
4,00fd7becb50ab7fddd284b633a68be4f,"[0, 5, 4, 1688, 6, 8, 2693, 5, 6, 8, 2077, 34,..."
...,...,...
1594,ff46d80e496dcf619510c8afe9a4696e,"[0, 7, 8, 1536, 5, 7, 13721, 5, 8, 4, 1875, 39..."
1595,ff73ba77cc519c9731001586b3cf56ad,"[0, 7, 11, 1685, 4, 5, 2718, 9, 11, 8, 1806, 3..."
1596,ff7de558c7e51213dcf10998411618fb,"[0, 5, 4, 1558, 5, 8, 6271, 5, 6, 8, 1633, 43,..."
1597,ff8ae4d9f46d819134f579453e2bb786,"[0, 8, 5, 1444, 7, 10, 2937, 10, 7, 4, 1717, 3..."


In [27]:
type(answ2['Result'])

pandas.core.series.Series

In [33]:
import statistics
def proper(row): #функция расчета дисперсии
    return statistics.variance(row['Result'])

In [34]:
answ2['answer'] = answ2.apply(proper,axis=1)
answ2.sort_values('answer')

,session_id,Result,answer
68,0b6ff2cdc9202e9da70dbb8a0e9bc6e9,"[0, 11, 6, 13, 12, 14, 1792, 32, 6, 5, 5, 7, 6...",3.098865e+05
1028,a3e5563f6259f7ea53e251e058b8af25,"[0, 7, 5, 5, 6, 4, 1749, 31, 6, 4, 5, 4, 3, 6,...",3.112763e+05
370,3ce3ef68895958bddf308c4a86d75f69,"[0, 12, 19, 15, 18, 16, 1938, 59, 8, 11, 6, 8,...",3.175900e+05
988,9e643e715f004a7c273a41c71c2d3777,"[0, 5, 4, 4, 8, 5, 1970, 34, 7, 5, 5, 4, 3, 6,...",3.219937e+05
420,445d8f304da4611c568ce7b98eeffbfe,"[0, 6, 4, 5, 7, 5, 1788, 33, 4, 4, 5, 5, 4, 8,...",3.444730e+05
...,...,...,...
1533,f4cd592c2ad8eb3db4d13376c46f9a49,"[0, 4, 4, 1610, 5, 5, 50277, 5, 6, 4, 1736, 32...",8.376208e+07
116,14b1efbc27f8a19ce38d42ba21e48410,"[0, 6, 4, 1599, 8, 9, 46955, 6, 7, 4, 1804, 34...",8.450890e+07
1431,e35e3228f80531fbc06572a0baf07c11,"[0, 5, 4, 1826, 5, 6, 64027, 15, 11, 18, 1895,...",1.568133e+08
734,7714ae1e594b3b4cb64cae7a4607bcce,"[0, 8, 5, 1650, 18, 5, 64135, 6, 8, 5, 1951, 3...",1.574404e+08


In [35]:
(full.loc[full['session_id'] == '4dcab2af4573ced8251fb7433174ba15']).head(3)

,forecastMarker,request_time,event_id,sender,session_id
0,,,,,
5738,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.203702,1,weekly_report.care,4dcab2af4573ced8251fb7433174ba15
5739,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.210420,29,weekly_report.care,4dcab2af4573ced8251fb7433174ba15
5740,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.214649,2,weekly_report.care,4dcab2af4573ced8251fb7433174ba15


## Самый большой разброс у session_id = 4dcab2af4573ced8251fb7433174ba15<br> 
### sender - weekly_report
#### Связано это может быть с большими объемом данных полученных за неделю и необходимым временем на его обработку

_

###  Дадим ответ на третий вопрос. 
#### Какая задача прогнозирования выполняется дольше всего,название задачи?

In [36]:
sd = ans1.loc[ans1['event_id'] == 31]
ans3 = ans1[ans1["session_id"].isin(sd["session_id"])]
ans3.head(6)

,forecastMarker,request_time,event_id,sender,session_id,timedelta,NumDays,Days,Time,Result
0,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.821083,1,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00,0,days,00:00:00,0
1,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.831013,29,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00.009930,0,days,00:00:00.009930,9
2,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:29.836547,2,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00.005534,0,days,00:00:00.005534,5
3,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.576231,3,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:01.739684,0,days,00:00:01.739684,1739
4,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.587533,5,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00.011302,0,days,00:00:00.011302,11
5,weekly_report.SAAS:new_visitors:utm_medium:pre...,2022-09-12 17:11:31.594359,6,weekly_report.saas,0035de8fad557cfe0b187f24b954195f,0 days 00:00:00.006826,0,days,00:00:00.006826,6


In [37]:
third_answer = pd.DataFrame(ans3.groupby('session_id')['timedelta']. sum ())
ta =third_answer.sort_values('timedelta')#Объеденим данные по сесси и посчитаем суммарное время, выведем самое большое время
ta.tail(1)

,timedelta
session_id,
4dcab2af4573ced8251fb7433174ba15,0 days 00:01:52.484473


In [38]:
third_answer.head(1)

,timedelta
session_id,
0035de8fad557cfe0b187f24b954195f,0 days 00:00:09.388732


In [39]:
(full.loc[full['session_id'] == '4dcab2af4573ced8251fb7433174ba15']).head(1)

,forecastMarker,request_time,event_id,sender,session_id
0,,,,,
5738,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.203702,1,weekly_report.care,4dcab2af4573ced8251fb7433174ba15


# weekly_report.care:count_of_jivo_SA  - <br>данная задача выполняется дольше остальных

_

# Дадим ответ на четверый вопрос.
## Перечислить forecastMarker которые не завершились успешно

In [40]:
sd5 = ans1.loc[ans1['event_id'] == 31]#Найдем евенты которые завершились удачно,удачно значит наступило 31 событие

sd5 #1594 session_id uniq

sdd5 = pd.DataFrame(full['session_id'].unique()) #1599 session_id uniq
ans5 = sdd5[~sdd5[0].isin(sd5["session_id"])]#Вычтем из всех сессий удачные 

In [41]:
spsk = []
for i in ans5[0]:
    n = full.loc[full['session_id'] == i]['forecastMarker']
    spsk.append(n.unique())
spsk1 = pd.DataFrame(spsk)
spsk1 #Выведем список неудачно завершившихся сессий


,0
0,None
1,weekly_report.SAAS:new_visitors
2,SaaS/RUS/turnover/targetMailRu
3,weekly_report.SAAS:turnover
4,weekly_report.SAAS:new_visitors:utm_medium:org...


## Выше предоставлен список forecastMarker которые не завершились успешно
### Из 1599 всего 5 не удавшихся

In [42]:
 # Be sure to close the connection
con.close()
print("соединение закрыто")

соединение закрыто


### Задание выполнил: Федоренко Вячеслав 
##### +79536416424